In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.api import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf

Function for data frame manipulations that are same for all data frames.

In [111]:
def df_manipulation(path):
    data_frame = pd.read_csv(path)
    data_frame = data_frame.rename(columns={"Name": "Date"})
    data_frame["Date"] = pd.to_datetime(data_frame["Date"], format="%d/%m/%Y")
    data_frame['Date'] = data_frame['Date'].dt.strftime('%Y-%m')
    data_frame.set_index('Date', inplace=True)
    return data_frame

Dependant variable data frame

In [112]:
dependentVariable = df_manipulation('data/depVariable.csv')

Monthly data

In [113]:
monthlyIndex = df_manipulation('data/Monthly.csv')

Change variable

In [114]:
changeVariables = df_manipulation('data/changeVariable.csv')

Smoothed 10 years earnings to price ratio

In [115]:
smoothedEP = df_manipulation('data/smoothedPE.csv')

Variance data frame, I don't use df_manipulation function because we need day information.

In [116]:
stockVariance = pd.read_csv('data/stockVar.csv')
stockVariance = stockVariance.rename(columns={"Name": "Date"})
stockVariance["Date"] = pd.to_datetime(stockVariance["Date"], format="%d/%m/%Y")
stockVariance.set_index('Date', inplace=True)

stockVariance = stockVariance.rename(columns={"JAPAN-DS Market - PRICE INDEX": "var"})

Net equity expansion

In [117]:
netIssue = df_manipulation('data/netEquExp.csv')

In [118]:
notCompVwretd = pd.DataFrame(dependentVariable["JAPAN-DS Market - TOT RETURN IND"].pct_change(1) - ((dependentVariable["JP OVERNIGHT UNCOLLATERISED CALL MONEY RATE (AVG.) NADJ"]/100)/12))

Excess market return following Li et al (2013) for regression: log(1+ret%) - log (1+tbill%)

In [119]:
dependentVariable["JAPAN-DS Market - TOT RETURN IND"] = np.log(dependentVariable["JAPAN-DS Market - TOT RETURN IND"]/dependentVariable["JAPAN-DS Market - TOT RETURN IND"].shift(1))

dependentVariable["JP OVERNIGHT UNCOLLATERISED CALL MONEY RATE (AVG.) NADJ"] = np.log(1+(dependentVariable["JP OVERNIGHT UNCOLLATERISED CALL MONEY RATE (AVG.) NADJ"]/100)/12)

dependentVariable["JAPAN-DS Market - TOT RETURN IND"] -= dependentVariable["JP OVERNIGHT UNCOLLATERISED CALL MONEY RATE (AVG.) NADJ"]

dependentVariable = dependentVariable.drop(columns=["JP OVERNIGHT UNCOLLATERISED CALL MONEY RATE (AVG.) NADJ"])
dependentVariable = dependentVariable.rename(columns={"JAPAN-DS Market - TOT RETURN IND": "totalReturnChange"})
dependentVariable = dependentVariable.dropna()


/Users/jessekeranen/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Help variables for valuation ratios

In [120]:
dividends = monthlyIndex["JAPAN-DS Market - PRICE INDEX"]/monthlyIndex["JAPAN-DS Market - TOT RETURN IND"]
price = dividends/monthlyIndex["JAPAN-DS Market - DIVIDEND YIELD"]*100
earnings = price/monthlyIndex["JAPAN-DS Market - PER"]
dividendPriceRatio = pd.DataFrame(monthlyIndex["JAPAN-DS Market - DIVIDEND YIELD"]/100)
dividendPriceRatio = dividendPriceRatio.rename(columns={dividendPriceRatio.columns[0]: "D/P"})
bookToMarketRatio = 1/pd.DataFrame(monthlyIndex["JAPAN-DS Market - PRICE/BOOK RATIO"])
bookToMarketRatio = bookToMarketRatio.rename(columns={"JAPAN-DS Market - PRICE/BOOK RATIO": "B/M"})

earningsToPriceRatio = 1/pd.DataFrame(monthlyIndex["JAPAN-DS Market - PER"])
earningsToPriceRatio = earningsToPriceRatio.rename(columns={"JAPAN-DS Market - PER": "E/P"})

Volatility and variance variables

In [121]:
volatilityIndex = pd.DataFrame(np.log(1 + changeVariables["NIKKEI STOCK AVERAGE VOLATILITY INDEX - PRICE INDEX"].diff() / changeVariables["NIKKEI STOCK AVERAGE VOLATILITY INDEX - PRICE INDEX"].shift(1)))

volatilityIndex = volatilityIndex.dropna()
volatilityIndex = volatilityIndex.rename(columns={volatilityIndex.columns[0]: "Vola"})

/Users/jessekeranen/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [122]:
stockVariance = pd.DataFrame((stockVariance["var"].diff() / stockVariance["var"].shift(1)).dropna())

In [123]:
stockVariance = stockVariance.groupby(stockVariance.index.to_period('M')).agg("var")["var"]
stockVariance.index = volatilityIndex.index

In [124]:
sDividends = smoothedEP["JAPAN-DS Market - PRICE INDEX"]/smoothedEP["JAPAN-DS Market - TOT RETURN IND"]
sPrice = sDividends/smoothedEP["JAPAN-DS Market - DIVIDEND YIELD"]*100
sEarnings = sPrice/smoothedEP["JAPAN-DS Market - PER"]
sEarnings = sEarnings.rolling(119).mean()
smoothedEP = sEarnings.dropna()

# Dates for December 1989 don't match in price and smoothedEP, so I change them manually.
price1 = price.copy()
price1.index.array[0] = '1989-12-29'

smoothedEP = (1/price1/smoothedEP).dropna()

smoothedEP = pd.DataFrame(smoothedEP)
smoothedEP = smoothedEP.rename(columns={smoothedEP.columns[0]: "E10/P"})

In [125]:
percentEquityIssuing = pd.DataFrame(monthlyIndex["JP STOCKS: PUBLIC OFFERINGS - AMOUNT RAISED CURN"]/(monthlyIndex["JP ISSUES: CORPORATE STRAIGHT BONDS CURN"] + monthlyIndex["JP STOCKS: PUBLIC OFFERINGS - AMOUNT RAISED CURN"]))
percentEquityIssuing = pd.DataFrame(percentEquityIssuing.fillna(0))
percentEquityIssuing = percentEquityIssuing.rename(columns={percentEquityIssuing.columns[0]: "Equis"})

inflationRate = pd.DataFrame(monthlyIndex["JP CPI: NATIONAL MEASURE - ANNUAL INFLATION RATE NADJ"]/100)
inflationRate = inflationRate.rename(columns={inflationRate.columns[0]: "Ifl"})

In [126]:
def change(x, column):
    return pd.DataFrame((x[column].diff() / x[column].shift(1)).dropna())

In [127]:
realMoneySupply = change(changeVariables, "JP MONEY SUPPLY: M2 (METHO-BREAK, APR. 2003) CURA")
realMoneySupply = realMoneySupply.rename(columns={realMoneySupply.columns[0]: "Rms"})

petroleumConsumption = change(changeVariables, "JP PETROLEUM: CONSUMPTION VOLN")
petroleumConsumption = petroleumConsumption.rename(columns={petroleumConsumption.columns[0]: "PCon"})
unemploymentRate = change(changeVariables, "JP UNEMPLOYMENT RATE (METHO BREAK OCT 2010) SADJ")
unemploymentRate = unemploymentRate.rename(columns={unemploymentRate.columns[0]: "Unem"})
industrialProduction = change(changeVariables, "JP INDUSTRIAL PRODUCTION - MINING & MANUFACTURING VOLA")
industrialProduction = industrialProduction.rename(columns={industrialProduction.columns[0]: "IProd"})
crudeOilPriceChange = change(changeVariables, "US REFINERS ACQUISITION COST OF DOM. & IMPORTED CRUDE OIL CURN")
crudeOilPriceChange = crudeOilPriceChange.rename(columns={crudeOilPriceChange.columns[0]: "C/P"})
crudeOilProduction = change(changeVariables, "WD CRUDE OIL PRODUCTION - WORLD VOLN")
crudeOilProduction = crudeOilProduction.rename(columns={crudeOilProduction.columns[0]: "C/O"})

In [128]:
mv = netIssue["JAPAN-DS Market - MARKET VALUE"]

temp9 = netIssue["JAPAN-DS Market - PRICE INDEX"].diff() / netIssue["JAPAN-DS Market - PRICE INDEX"].shift(1)
netIssue = netIssue["JAPAN-DS Market - MARKET VALUE"] - netIssue["JAPAN-DS Market - MARKET VALUE"].shift(1)*(1+temp9)

netIssue = (netIssue.rolling(window=12, min_periods=3).sum()).dropna()
netIssue = (netIssue/mv).dropna()
netIssue = pd.DataFrame(netIssue)
netIssue = netIssue.rename(columns={netIssue.columns[0]: "Ntis"})

In [129]:
netPY = (monthlyIndex["JAPAN-DS Market - DIVIDEND YIELD"]/100) + (((changeVariables["JAPAN-DS Market - MARKET VALUE"].shift(1) * (changeVariables["JAPAN-DS Market - PRICE INDEX"]/changeVariables["JAPAN-DS Market - PRICE INDEX"].shift(1))) - changeVariables["JAPAN-DS Market - MARKET VALUE"])/changeVariables["JAPAN-DS Market - MARKET VALUE"])
netPY = pd.DataFrame((np.log(0.1 + netPY)).dropna())
netPY = netPY.rename(columns={netPY.columns[0]: "Ndy"})

/Users/jessekeranen/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [130]:
def rollingmean(length):
    temp = dependentVariable.rolling(length).mean().shift(-length)
    temp = temp.rename(columns={temp.columns[0]: ("K" + str(length))})
    return pd.DataFrame(temp)

In [131]:
mk12 = rollingmean(12)
mk24 = rollingmean(24)
mk36 = rollingmean(36)
mk48 = rollingmean(48)

Fama and French factors

In [ ]:
famafrench = pd.read_csv("data/Japan_5_Factors.csv")
famafrench["Date"] = pd.to_datetime(famafrench["Date"], format="%Y%m")
famafrench['Date'] = famafrench['Date'].dt.strftime('%Y-%m')
famafrench.set_index('Date', inplace=True)
famafrench = famafrench.loc[:].div(100)
famafrench = famafrench.drop(columns={"RF"})

mom = pd.read_csv("data/Japan_MOM_Factor.csv")
mom['Date'] = pd.to_datetime(mom["Date"], format="%Y%m")
mom['Date'] = mom['Date'].dt.strftime('%Y-%m')
mom.set_index('Date', inplace=True)
mom = mom.loc[:].div(100)

In [ ]:
finalmonthlyOnlyIV = dividendPriceRatio.merge(bookToMarketRatio, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(earningsToPriceRatio, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(smoothedEP, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(percentEquityIssuing, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(netIssue, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(netPY, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(inflationRate, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(realMoneySupply, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(crudeOilPriceChange, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(crudeOilProduction, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(petroleumConsumption, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(industrialProduction, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(unemploymentRate, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(stockVariance, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(volatilityIndex, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(famafrench, on="Date")
finalmonthlyOnlyIV = finalmonthlyOnlyIV.merge(mom, on="Date")

In [ ]:
finalmonthlyOnlyIV

In [ ]:
dependentVariable["totalReturnChange"] = dependentVariable["totalReturnChange"].shift(-1)

dependentVariable

In [ ]:
dependentVariable = dependentVariable.dropna()
dependentVariable = dependentVariable.rename(columns={dependentVariable.columns[0]: "K1"})
finalIndexMonthly = finalmonthlyOnlyIV.merge(dependentVariable, on='Date')
finalIndexMonthly = finalIndexMonthly.merge(mk12, on='Date')
finalIndexMonthly = finalIndexMonthly.merge(mk24, on='Date')
finalIndexMonthly = finalIndexMonthly.merge(mk36, on='Date')
finalIndexMonthly = finalIndexMonthly.merge(mk48, on='Date')

In [ ]:
finalIndexMonthly

Some plotting and descriptive statistics

In [ ]:
plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=1)

for i, col in enumerate(finalIndexMonthly.columns):
    ax = plt.subplot(10, 3, i + 1)

    finalIndexMonthly[col].hist(ax=ax)

    ax.set_title(col.upper())
    ax.set_xlabel("")

In [ ]:
plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=1)

for i, col in enumerate(finalIndexMonthly.columns):
    ax = plt.subplot(10, 3, i + 1)

    finalIndexMonthly[col].plot(ax=ax)

    ax.set_title(col.upper())
    ax.set_xlabel("")

In [ ]:
plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=1)

for i, col in enumerate(finalIndexMonthly.columns):
    ax = plt.subplot(10, 3, i + 1)

    finalIndexMonthly.plot.scatter(x=col, y="K1", ax=ax)

    ax.set_title(col.upper())
    ax.set_xlabel("")

In [ ]:
finalIndexMonthly.describe()

In [ ]:
#addingsStarts
def addStars(data):
    if data <= 0.01:
        return str(data) + "***"
    elif data <= 0.05:
        return str(data) + "**"
    elif data <= 0.1:
        return str(data) + "*"
    else:
        return str(data)
  

Univariate insample results

In [ ]:
# I first make a listoflist to dynamically fill the table with strings and integers
# afterwards I convert it to a DataFrame
listoflist = list()

# For every independent variable
for i in finalmonthlyOnlyIV.columns:
    listoflist.append(["Variable", i, "-", "-", "-"])
    #in the end i calculate the average coefficient of all the time periods
    averageOfCoefficients = list()
    
    #for all the time periods: K1, K12, K24, K36, K48
    for j in finalIndexMonthly.iloc[:,-5:].columns:
        olsString = '' + j + ' ~ 1 + Q("' + i + '")'

        #I fit with newey west standard errors and lag of 1
        ols = smf.ols(olsString, data = finalIndexMonthly).fit(cov_type = 'HAC', cov_kwds={'maxlags':1})

        #add Stars to pvalue according to their significance levels
        pval = addStars(round(ols.pvalues[1],3))

        #period, coefficient of variable, newey west tstat, pvalue, r^2 adj value
        listoflist.append([j, round(ols.params[1],2), round(ols.tvalues[1],2), pval, round(ols.rsquared_adj,2)])

        averageOfCoefficients.append(ols.params[1])
    listoflist.append(["Avg", round(np.mean(averageOfCoefficients),2), "-", "-", "-"])

resultsUV = pd.DataFrame(listoflist)
resultsUV.columns = ["Time Horizon", "Coefficient (b)", "NW t-stat", "pval", "R^2 adj"]

#set first column as index of dataframe
resultsUV = resultsUV.set_index(resultsUV.iloc[:,0].values)
resultsUV = resultsUV.drop(resultsUV.columns[0], axis=1)     
resultsUV

Multivariate insample results

In [ ]:
test = finalIndexMonthly.iloc[:, :-5]
features = list(test.columns)
features.insert(0, "const")

timePeriod = "K1" #change this to K12, K24, K36, K48
olsString = timePeriod + ' ~ 1'
for i in finalmonthlyOnlyIV.columns:
    olsString = olsString + ' + Q("' + i + '")'

#I fit with newey west standard errors and lag of 1
ols = smf.ols(olsString, data = finalIndexMonthly).fit(cov_type = 'HAC', cov_kwds={'maxlags':1})

print(ols.summary(xname=features))

Correlations

In [ ]:
correlations = test.corr()

correlations

VIF

In [ ]:
vif_df = finalmonthlyOnlyIV
vif = pd.DataFrame()
vif["Variable"] = vif_df.columns
vif["VIF"] = [variance_inflation_factor(vif_df.values, column) for column in range(len(vif_df.columns))]

maxim = vif.loc[vif["VIF"] == vif["VIF"].max()].reset_index(drop=True)

maxim

while maxim["VIF"][0] > 10:
    vif_df = vif_df.drop(columns={maxim["Variable"][0]})
    vif = pd.DataFrame()
    vif["Variable"] = vif_df.columns
    vif["VIF"] = [variance_inflation_factor(vif_df.values, column) for column in range(len(vif_df.columns))]
    maxim = vif.loc[vif["VIF"] == vif["VIF"].max()].reset_index(drop=True)

vif

In [ ]:
timePeriod = "K48" #change this to K12, K24, K36, K48
olsString = timePeriod + ' ~ 1'
for i in vif_df.columns:
    olsString = olsString + ' + Q("' + i + '")'

#I fit with newey west standard errors and lag of 1
vif_ols = smf.ols(olsString, data = finalIndexMonthly).fit(cov_type = 'HAC', cov_kwds={'maxlags':1})

print(vif_ols.summary())

Utility Calculation

In [ ]:
indexTemp = finalIndexMonthly.index
#historical average market return
hmean = list()
for i in indexTemp:
    hmean.append(finalIndexMonthly.loc[:,"K1"].loc["1990-11":i].mean())

#historical average market return
rhmean = pd.Series(hmean[finalIndexMonthly.index.get_loc("2001-12"):-1])



#utility gain
riskAversion = 3
varianceUT = finalIndexMonthly.loc[:,"K1"]*100
#utilityDF
varianceUT = varianceUT.rolling(len(varianceUT),min_periods=12*10).var().loc["2002-01":]
rhmean.index = varianceUT.index
wVariable = (rhmean*100).div(varianceUT)*(1/riskAversion)


#restrict wVariable to max 0% or 150%
wVariable
for i in wVariable.index:
    if wVariable[i] < 0:
        wVariable[i] = 0
    elif wVariable[i] > 1.5:
        wVariable[i] = 1.5
    else:
        True
        #do nothing
historicUtility = wVariable.mean()-(1/2*riskAversion*wVariable.var())

Out of sample predictions

In [ ]:
# Add constants to the model
finalmonthlyOnlyIVConst = finalmonthlyOnlyIV.copy(deep=True)
finalmonthlyOnlyIVConst.insert(0, "const", 1)

finalIndexMonthlyConst = finalIndexMonthly.copy(deep=True)
finalIndexMonthlyConst.insert(0, "const", 1)
finalIndexMonthlyConst

In [ ]:
dates = finalIndexMonthly.loc["2002-01":].index
results = np.empty(shape=(0, 4))
predictions = []

for date in dates:
    out_of_sample = finalIndexMonthlyConst.loc[date].iloc[:-5]

    df = finalIndexMonthlyConst.loc[:date].iloc[:-1, :-4]
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1:]
    ols = sm.OLS(y, X).fit()
    temp = np.matrix([[date] * len(ols.params)] + [ols.params.tolist()] + [ols.tvalues.tolist()] + [ols.params.index.tolist()])

    temp = np.transpose(temp)
    results = np.vstack((results, temp))

    prediction = [date, ols.predict(out_of_sample)[0]]
    predictions.append(prediction) #same as oospread

pred_df = pd.DataFrame(predictions)
pred_df = pred_df.rename(columns={0: "Date", 1: "Pred"})
pred_df.set_index('Date', inplace=True)

results = pd.DataFrame(results)
results = results.rename(columns={0: "Date", 1: "Coef", 2: "T-val", 3: "Variable"})
results.set_index("Date", inplace=True)

results[["Coef", "T-val"]] = results[["Coef", "T-val"]].apply(pd.to_numeric)
results = results.round(2)
#results

In [ ]:
plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=1)

for i, variable in enumerate(features[0:]):
    ax = plt.subplot(10, 3, i + 1)

    results[results["Variable"] == variable].plot(ax=ax)

    ax.set_title(variable.upper())
    ax.get_legend().remove()
    ax.set_xlabel("")

plt.legend(loc="lower center", bbox_to_anchor=(1.2, 0))
plt.show()

In [ ]:
cols = vif_df.columns
cols = cols.insert(0, "const")
cols = cols.insert(0, "K1")

dates = finalIndexMonthly.loc["2002-01":].index
results_vif = np.empty(shape=(0, 4))
predictions_vif = []

for date in dates:
    out_of_sample = finalIndexMonthlyConst.loc[date, cols[1:]]

    df = finalIndexMonthlyConst.loc[:date, cols].iloc[:-1]
    X = df.iloc[:, 1:]
    y = df.iloc[:, 0]
    ols = sm.OLS(y, X).fit()
    temp = np.matrix([[date] * len(ols.params)] + [ols.params.tolist()] + [ols.tvalues.tolist()] + [ols.params.index.tolist()])

    temp = np.transpose(temp)
    results_vif = np.vstack((results_vif, temp))

    prediction_vif = [date, ols.predict(out_of_sample)[0]]
    predictions_vif.append(prediction_vif) #same as oospread

pred_df_vif = pd.DataFrame(predictions_vif)
pred_df_vif = pred_df_vif.rename(columns={0: "Date", 1: "Pred_vif"})
pred_df_vif.set_index('Date', inplace=True)

results_vif = pd.DataFrame(results_vif)
results_vif = results_vif.rename(columns={0: "Date", 1: "Coef", 2: "T-val", 3: "Variable"})
results_vif.set_index("Date", inplace=True)

results_vif[["Coef", "T-val"]] = results_vif[["Coef", "T-val"]].apply(pd.to_numeric)
results_vif = results_vif.round(2)

In [ ]:
plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=1)

for i, variable in enumerate(cols[1:]):
    ax = plt.subplot(10, 3, i + 1)

    results_vif[results_vif["Variable"] == variable].plot(ax=ax)

    ax.set_title(variable.upper())
    ax.get_legend().remove()
    ax.set_xlabel("")

plt.legend(loc="lower center", bbox_to_anchor=(1.2, 0))
plt.show()

Calculation of R^2

In [ ]:
realReturns = finalIndexMonthly.loc["2002-01":,"K1"]
realReturns

In [ ]:
dates = finalIndexMonthly.loc["2002-01":"2020-08"].index
anzahlIV = len(finalmonthlyOnlyIV.columns)
resultsForecastUV = np.zeros((anzahlIV,4))

figuresOosUV = list()

#real return
realReturns = finalIndexMonthly.loc["2002-01":,"K1"]
realReturns.index = rhmean.index

rapachTabelle = np.zeros((len(finalIndexMonthly.loc["2002-01":,]),anzahlIV))
rapachCounter = 0

for i, IV in enumerate(finalmonthlyOnlyIV.columns):
    oospread = list()
    oospreadCampbell = list()
    for counter, j in enumerate(dates):
        X = finalIndexMonthly.loc[:j,IV].iloc[:-1]
        y = finalIndexMonthly.loc[:j,"K1"].iloc[:-1]
        X = sm.add_constant(X, has_constant = 'add')
        ols = sm.OLS(y, X).fit()
        olsTemp = ols.params[0] + ols.params[1]*finalIndexMonthly.loc[j][i]
        oospread.append(olsTemp)

        rapachTabelle[rapachCounter, i] = olsTemp
        rapachCounter += 1

        temp = finalIndexMonthly.loc[:j,IV].mean()
        temp2 = None
        ols = sm.OLS(y, X).fit()
        if (temp < 0 and ols.params[1] < 0 ) or (temp > 0 and  ols.params[1] > 0):
            temp2 = olsTemp
        else:
            temp2 = ols.params[0] * ols.params[1]*finalIndexMonthly.loc[j,IV]
        if temp2 < 0:
            temp2 = 0
        else:
            temp2 = temp2
        oospreadCampbell.append(temp2)
    rapachCounter = 0

    #R^2 os
    oospread = pd.Series(oospread)
    oospread.index = rhmean.index
    rNormalR2os = 1 -(np.square(realReturns-oospread)).sum()/(np.square(realReturns-rhmean)).sum()
    resultsForecastUV[i,0] = round(rNormalR2os*100,2)

    #adjusted MSPE following Clark and West (2007) to get pval
    rAdjustMspe = np.square(realReturns-rhmean) - (np.square(realReturns-oospread) - np.square(rhmean - oospread))
    rAdjustMspeTest_statistic, rAdjustMspeTest_pval = stats.ttest_1samp(rAdjustMspe, popmean = 0,alternative = 'greater')
    resultsForecastUV[i,1] = round(rAdjustMspeTest_pval,3)

    #R^2 os Campbell
    rNormalMspeCampbell = 1-(np.square(realReturns-oospreadCampbell)).sum()/(np.square(realReturns-rhmean)).sum()
    resultsForecastUV[i,2] = round(rNormalMspeCampbell*100,2)

    #data for figures (normal ones)
    sqm = list()
    for k in oospread.index:
        sqm.append(np.square(realReturns.loc[:k] - rhmean.loc[:k]).sum() - np.square(realReturns.loc[:k] - oospread.loc[:k]).sum())
    figuresOosUV.append(sqm)

    #utility gain: here portfolio with forecast
    varianceUT.index = oospread.index
    wVariable = (oospread*100).div(varianceUT)*(1/riskAversion)
    #restrict wVariable to max 0% or 150%
    wVariable
    for a in wVariable.index:
        if wVariable[a] < 0:
            wVariable[a] = 0
        elif wVariable[a] > 1.5:
            wVariable[a] = 1.5
        else:
            True
            #do nothing
    utilityPred = wVariable.mean()-(1/2*riskAversion*wVariable.var())
    gain = (utilityPred - historicUtility)*1200
    resultsForecastUV[i,3] = round(gain,2)

resultsForecastUV = pd.DataFrame(resultsForecastUV)
resultsForecastUV.columns = ["R^2OS", "pval", "signRE", "Ugain"]
resultsForecastUV.index = finalmonthlyOnlyIV.columns
#resultsForecastUV

In [ ]:
figuresOosUVDF = pd.DataFrame(np.transpose(figuresOosUV))
figuresOosUVDF = figuresOosUVDF.set_index(dates)
figuresOosUVDF.columns = finalmonthlyOnlyIV.columns

plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=1)

for i, col in enumerate(figuresOosUVDF.columns):
    ax = plt.subplot(10, 3, i + 1)

    figuresOosUVDF[col].plot(ax=ax)

    ax.set_title(col.upper())
    ax.set_xlabel("")

Multivariate Out of Sample approach

In [ ]:
tempRapach = rapachTabelle.copy()
rapachTabelle = pd.DataFrame(rapachTabelle)
rapachTabelle.round(5)
rapachTabelle.columns = finalmonthlyOnlyIV.columns

#mean
rapachTabelle["mean"] = rapachTabelle.mean(axis=1)
#median
rapachTabelle["median"] = rapachTabelle.median(axis = 1)

#trimmed mean
tempRapach[np.arange(len(rapachTabelle)), rapachTabelle.values.argmax(1)] = 0
tempRapach[np.arange(len(rapachTabelle)), rapachTabelle.values.argmin(1)] = 0
tempRapach = pd.DataFrame(tempRapach)
rapachTabelle["trimmedMean"] = tempRapach.sum(axis = 1) / (anzahlIV - 2)

In [ ]:
# Set index
rapachTabelle = rapachTabelle.set_index(dates)
rapachTabelle

In [ ]:
###DMPSE combination
discountFactor1 = 0.9
discountFactor2 = 1.0
m = 134
t = len(finalIndexMonthly.index)


weightsV1 = list() #discount factor 1
weightsV2 = list() #discount factor 2

#calculating phi eq 13
for i in finalmonthlyOnlyIV.columns:
    tempComb = pd.Series(rapachTabelle.loc[:,i])
    tempComb.index = realReturns.index
    weightsV1.append((discountFactor1**np.arange(t-m-1,t-1-(t-1)-1,-1) * np.square(realReturns - tempComb)).sum())
    weightsV2.append((discountFactor2**np.arange(t-m-1,t-1-(t-1)-1,-1) * np.square(realReturns - tempComb)).sum())

#calculating omega eq 12
tempComb1 = weightsV1
tempComb2 = weightsV2
tmp1 = list()
tmp2 = list()
for counter, value in enumerate(weightsV1):
    tmp1.append(value/sum(weightsV1))
    tmp2.append(weightsV2[counter]/sum(weightsV2))

#returns are formed with corresponding weights
tabelle1 = rapachTabelle.copy()
tabelle2 = rapachTabelle.copy()
for i in rapachTabelle.index:
    for counter, j in enumerate(rapachTabelle.columns[:-3]):
        tabelle1.loc[i,j] = tmp1[counter]*rapachTabelle.loc[i,j]
        tabelle2.loc[i,j] = tmp2[counter]*rapachTabelle.loc[i,j]

rapachTabelle["combDMPSE1"] = np.sum(tabelle1, axis = 1)
rapachTabelle["combDMPSE2"] = np.sum(tabelle2, axis = 1)

Logistic regression

In [ ]:
results_log = np.empty(shape=(0, 4))
predictions_log = []

log_df = finalIndexMonthlyConst.iloc[:, :-4].copy()
log_df.loc[log_df["K1"] < 0, "K1" ] = 0
log_df.loc[log_df["K1"] > 0, "K1" ] = 1

for date in dates:
    out_of_sample = log_df.loc[date].iloc[:-1]
    df = log_df.loc[:date]
    X = df.iloc[:, :-1].iloc[:-1]
    y = df.iloc[:, -1:].iloc[:-1]
    glm = sm.GLM(y, X, family=sm.families.Binomial()).fit()
    temp = np.matrix([[date] * len(glm.params)] + [glm.params.tolist()] + [glm.tvalues.tolist()] + [glm.params.index.tolist()])

    temp = np.transpose(temp)
    results_log = np.vstack((results_log, temp))

    prediction = [date, glm.predict(out_of_sample)[0]]
    predictions_log.append(prediction)

pred_df_log = pd.DataFrame(predictions_log)
pred_df_log = pred_df_log.rename(columns={0: "Date", 1: "Pred_log"})
pred_df_log.set_index('Date', inplace=True)

results_log = pd.DataFrame(results_log)
results_log = results_log.rename(columns={0: "Date", 1: "Coef", 2: "T-val", 3: "Variable"})
results_log.set_index("Date", inplace=True)

results_log[["Coef", "T-val"]] = results_log[["Coef", "T-val"]].apply(pd.to_numeric)
results_log = results_log.round(2)

pred_df_log.loc[pred_df_log["Pred_log"] < 0.5, "Pred_log" ] = 0
pred_df_log.loc[pred_df_log["Pred_log"] >= 0.5, "Pred_log" ] = 1

pred_df_log = pred_df_log.merge(log_df["K1"], on="Date")
pred_df_log

In [ ]:
confusion_matrix = pd.crosstab(pred_df_log['K1'], pred_df_log['Pred_log'], rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)

In [ ]:
# I make dataframe from rhmean that I can merge it to df2
rhmeanDF = pd.DataFrame(rhmean)
rhmeanDF = rhmeanDF.set_index(dates)
rhmeanDF = rhmeanDF.rename(columns={0: "rhmean"})

# Final df2 contains rhmean, realized return and "all" multivariate predictions
df2 = pd.DataFrame(finalIndexMonthly["K1"]).merge(rhmeanDF, on="Date")
df2 = df2.merge(pred_df, on="Date")
df2 = df2.merge(rapachTabelle.iloc[:, -5:], on="Date")
df2 = df2.merge(pred_df_log["Pred_log"], on="Date")
df2 = df2.merge(pred_df_vif, on="Date")

# Make binary estimate continious
df2["Pred_log"] = df2["Pred_log"] * df2["rhmean"]

resultsForecastMV = np.zeros((len(df2.columns) - 2, 3))
figuresOosMV = list()

varianceUT.index = df2.index

i = 0
row_names = []

for prediction in df2.iloc[:,2:]:
    #R^2 os
    rNormalR2os = 1 - ((np.square(df2["K1"] - df2[prediction])).sum()/(np.square(df2["K1"] - df2["rhmean"])).sum())

    resultsForecastMV[i,0] = round(rNormalR2os*100,2)

    #adjusted MSPE following Clark and West (2007) to get pval
    rAdjustMspe = np.square(df2["K1"]-df2["rhmean"]) - (np.square(df2["K1"]-df2[prediction]) - np.square(df2["rhmean"] - df2[prediction]))
    rAdjustMspeTest_statistic, rAdjustMspeTest_pval = stats.ttest_1samp(rAdjustMspe, popmean = 0, alternative = 'greater')

    resultsForecastMV[i,1] = round(rAdjustMspeTest_pval, 3)


    sqm = list()
    for k in df2.index:
        sqm.append(np.square(df2.loc[:k, "K1"] - df2.loc[:k, "rhmean"]).sum() - np.square(df2.loc[:k, "K1"] - df2.loc[:k, prediction]).sum())
    figuresOosMV.append(sqm)


    wVariableKIT = (df2[prediction]*100).div(varianceUT)*(1/riskAversion)

    #restrict wVariable to max 0% or 150%
    wVariableKIT
    for a in wVariableKIT.index:
        if wVariableKIT[a] < 0:
            wVariableKIT[a] = 0
        elif wVariableKIT[a] > 1.5:
            wVariableKIT[a] = 1.5
        else:
            True
            #do nothing

    utilityPredKIT = wVariableKIT.mean()-(1/2*riskAversion*wVariableKIT.var())
    gainKIT = (utilityPredKIT - historicUtility)*1200
    resultsForecastMV[i,2] = round(gainKIT,2)
    row_names.append(prediction)
    i += 1

resultsForecastMV = pd.DataFrame(resultsForecastMV)
resultsForecastMV.columns = ["R^2OS", "pval", "Ugain"]
resultsForecastMV.index = row_names

resultsForecastMV

In [ ]:
figuresOosMVDF = pd.DataFrame(np.transpose(figuresOosMV))
figuresOosMVDF = figuresOosMVDF.set_index(dates)
figuresOosMVDF.columns = df2.columns[2:]


plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=1)

for i, col in enumerate(figuresOosMVDF.columns):
    ax = plt.subplot(4, 2, i + 1)

    figuresOosMVDF[col].plot(ax=ax)

    ax.set_title(col.upper())
    ax.set_xlabel("")